In [1]:
import os, gc
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2"
import tensorflow as tf
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
print('TensorFlow version =',tf.__version__)

2024-12-11 13:22:32.757951: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-11 13:22:32.758041: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-11 13:22:32.760175: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-11 13:22:32.770318: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-11 13:22:36.396641: W tensorflow/compiler/tf2

TensorFlow version = 2.15.1


In [2]:
import importlib
from UNet_HCRF_src.utility import UNet_layer
from UNet_HCRF_src.UNet_HCRF_model import unet_global


# USE MULTIPLE GPUS
gpus = tf.config.list_physical_devices('GPU')
if len(gpus)<=1: 
    strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    print(f'Using {len(gpus)} GPU')
else: 
    strategy = tf.distribute.MirroredStrategy()
    print(f'Using {len(gpus)} GPUs')

Using 0 GPU


In [3]:
df = pd.read_csv('./dataset/train.csv')
TARGETS = df.columns[-6:]

train = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_id':'first','spectrogram_label_offset_seconds':'min'})
train.columns = ['spectrogram_id','spec_min']

tmp = df.groupby('eeg_id')[['spectrogram_id','spectrogram_label_offset_seconds']].agg(
    {'spectrogram_label_offset_seconds':'max'})
train['spec_max'] = tmp

tmp = df.groupby('eeg_id')[['spectrogram_id', 'eeg_label_offset_seconds']].agg(
    {'eeg_label_offset_seconds':'min' }
)
train['eeg_min'] = tmp

tmp = df.groupby('eeg_id')[['spectrogram_id', 'eeg_label_offset_seconds']].agg(
    {'eeg_label_offset_seconds':'max' }
)
train['eeg_max'] = tmp

tmp = df.groupby('eeg_id')[['spectrogram_id', 'eeg_label_offset_seconds']].agg(
    {'eeg_label_offset_seconds':'mean' }
)
train['eeg_mean'] = tmp

tmp = df.groupby('eeg_id')[['patient_id']].agg('first')
train['patient_id'] = tmp

tmp = df.groupby('eeg_id')[TARGETS].agg('sum')
train['expert_numner'] = tmp.sum(axis=1)
for t in TARGETS:
    train[t] = tmp[t].values

y_data = train[TARGETS].values
y_data = y_data / y_data.sum(axis=1,keepdims=True)
train[TARGETS] = y_data

tmp = df.groupby('eeg_id')[['expert_consensus']].agg('first')
train['target'] = tmp

train = train.reset_index()
print('Train non-overlapp eeg_id shape:', train.shape )
train.head()
####################

TARGETS = train.columns[-7:-1]
print('Train shape:', train.shape)
print('Targets', list(TARGETS))
print('Unique EEG id:', train.eeg_id.unique().shape)
print('Spectrogram id:', train.spectrogram_id.unique().shape)
print('Unique EEG_Spectrogram id:', train.groupby(['eeg_id', 'spectrogram_id']).agg('first').shape[0])
# tmp_index = train.eeg_label_offset_seconds > train.spectrogram_label_offset_seconds
# print('Eeg offset time longer: ', train[tmp_index].shape[0])
# tmp_index = train.eeg_label_offset_seconds < train.spectrogram_label_offset_seconds
# print('Spectrogram offset time longer: ', train[tmp_index].shape[0])
tmp = train.groupby('eeg_id').apply(lambda x: len(np.unique(x['spectrogram_id'].values)))
print(f'Non-unique spectrogram id upper bound:{tmp.max()}')
tmp = train.groupby('spectrogram_id').apply(lambda x: len(np.unique(x['eeg_id'].values)))
print(f'Non-unique eeg id upper bound:{tmp.max()}')
# pd.DataFrame.groupby()
print(f'Over 10 experts rated records:{train[train.expert_numner > 10].shape}')
train.head()

Train non-overlapp eeg_id shape: (17089, 16)
Train shape: (17089, 16)
Targets ['seizure_vote', 'lpd_vote', 'gpd_vote', 'lrda_vote', 'grda_vote', 'other_vote']
Unique EEG id: (17089,)
Spectrogram id: (11138,)
Unique EEG_Spectrogram id: 17089
Non-unique spectrogram id upper bound:1
Non-unique eeg id upper bound:107
Over 10 experts rated records:(10143, 16)


,eeg_id,spectrogram_id,spec_min,spec_max,eeg_min,eeg_max,eeg_mean,patient_id,expert_numner,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote,target
0,568657,789577333,0.0,16.0,0.0,16.0,8.500000,20654,48,0.0,0.000000,0.25,0.000000,0.166667,0.583333,Other
1,582999,1552638400,0.0,38.0,0.0,38.0,17.454545,20230,154,0.0,0.857143,0.00,0.071429,0.000000,0.071429,LPD
2,642382,14960202,1008.0,1032.0,0.0,24.0,12.000000,5955,2,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other
3,751790,618728447,908.0,908.0,0.0,0.0,0.000000,38549,1,0.0,0.000000,1.00,0.000000,0.000000,0.000000,GPD
4,778705,52296320,0.0,0.0,0.0,0.0,0.000000,40955,2,0.0,0.000000,0.00,0.000000,0.000000,1.000000,Other


In [ ]:
%%time
import importlib
import utility
importlib.reload(utility)
import pandas as pd
directory_path = 'dataset/non_overlap_eeg_spec/'
PATH = 'dataset/train_eegs/'
DISPLAY = 4
EEG_info = train[['eeg_id', 'eeg_min', 'eeg_max']].values
all_eegs = {}
READ_EGG = False
SAVE_EGG_SPEC = False
if READ_EGG:
    for i,eeg_info in enumerate(EEG_info):
        # eeg_id, middle = int(eeg_info[0]), (eeg_info[1] + eeg_info[2]) // 2
        eeg_id, middle = int(eeg_info[0]), eeg_info[1]
        if (i%100==0)&(i!=0): print(i,', ',end='')
        # CREATE SPECTROGRAM FROM EEG PARQUET
        img = spectrogram_from_eeg(f'{PATH}{eeg_id}.parquet', i<DISPLAY, off_set= middle)
        
        # SAVE TO DISK
        if i==DISPLAY:
            print(f'Creating and writing {EEG_info.shape[0]} spectrograms to disk... ',end='')
        np.save(f'{directory_path}{eeg_id}',img)
        all_eegs[eeg_id] = img
    if SAVE_EGG_SPEC:
    # SAVE EEG SPECTROGRAM DICTIONARY
        np.save('mean_non_overlap_eeg_specs_non_denoise',all_eegs)
else:
    all_eegs = np.load('dataset/non_overlap_eeg_specs_non_denoise.npy', allow_pickle=True).item()

In [ ]:
%%time
READ_SPEC_FILES = False

# READ ALL SPECTROGRAMS
PATH = './dataset/train_spectrograms/'
files = os.listdir(PATH)
print(f'There are {len(files)} spectrogram parquets')

if READ_SPEC_FILES:    
    spectrograms = {}
    for i,f in enumerate(files):
        if i%100==0: print(i,', ',end='')
        tmp = pd.read_parquet(f'{PATH}{f}')
        name = int(f.split('.')[0])
        spectrograms[name] = tmp.iloc[:,1:].values
else:
    spectrograms = np.load('./dataset/spec.npy',allow_pickle=True).item()

### Dataloader for training

In [ ]:
import albumentations as albu
TARS = {'Seizure':0, 'LPD':1, 'GPD':2, 'LRDA':3, 'GRDA':4, 'Other':5}
TARS2 = {x:y for y,x in TARS.items()}

class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, batch_size=32, shuffle=False, augment=False, mode='train',
                 specs = spectrograms, eeg_specs = all_eegs):

        self.data = data
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.augment = augment
        self.mode = mode
        self.specs = specs
        self.eeg_specs = eeg_specs
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        ct = int( np.ceil( len(self.data) / self.batch_size ) )
        return ct

    def __getitem__(self, index):
        'Generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        X, y = self.__data_generation(indexes)
        if self.augment: X = self.__augment_batch(X)
        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange( len(self.data) )
        if self.shuffle: np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'

        X = np.zeros((len(indexes),128,256,8),dtype='float32')
        y = np.zeros((len(indexes),6),dtype='float32')
        img = np.ones((128,256),dtype='float32')

        for j,i in enumerate(indexes):
            row = self.data.iloc[i]
            if self.mode=='test':
                r = 0
            else:
                r = int( (row['spec_min'] + row['spec_max'])//4 )

            for k in range(4):
                # EXTRACT 300 ROWS OF SPECTROGRAM
                img = self.specs[row.spectrogram_id][r:r+300,k*100:(k+1)*100].T

                # LOG TRANSFORM SPECTROGRAM
                img = np.clip(img,np.exp(-4),np.exp(8))
                img = np.log(img)

                # STANDARDIZE PER IMAGE
                ep = 1e-6
                m = np.nanmean(img.flatten())
                s = np.nanstd(img.flatten())
                img = (img-m)/(s+ep)
                img = np.nan_to_num(img, nan=0.0)

                # CROP TO 256 TIME STEPS
                X[j,14:-14,:,k] = img[:,22:-22] / 2.0

            # EEG SPECTROGRAMS
            img = self.eeg_specs[row.eeg_id]
            X[j,:,:,4:] = img

            if self.mode!='test':
                y[j,] = row[TARGETS]

        return X,y

    def __random_transform(self, img):
        composition = albu.Compose([
            albu.HorizontalFlip(p=0.5),
            #albu.CoarseDropout(max_holes=8,max_height=32,max_width=32,fill_value=0,p=0.5),
        ])
        return composition(image=img)['image']

    def __augment_batch(self, img_batch):
        for i in range(img_batch.shape[0]):
            img_batch[i, ] = self.__random_transform(img_batch[i, ])
        return img_batch

### Build global unet model

In [ ]:
from tensorflow.keras.layers import (
    Layer,
    Conv2D,
    UpSampling2D,
    AveragePooling2D,
    LeakyReLU,
    Input,
    concatenate,
    BatchNormalization,
    Dense,
    Dropout,
    Activation,
    Add,
    GlobalAveragePooling2D,
)
from tensorflow.keras.models import Model
from crfasrnn_keras.src.crfrnn_layer import CrfRnnLayer
from crfasrnn_keras.src.hidden_crfnn_layer import HiddenCrfRnnLayer

class UNet_layer(Layer):
    def __init__(self, input_shape, num_classes=6, num_features=32, blocks=[1, 2, 4, 8, 16], trainable=False, **kwargs):
        super(UNet_layer, self).__init__(**kwargs)

        # Save parameters
        self.input_shape_ = input_shape
        self.num_classes = num_classes
        self.num_features = num_features
        self.blocks = blocks
        self.trainable = trainable

        # Define down-sampling layers
        self.down_stack = []
        for i, block in enumerate(self.blocks):
            self.down_stack.append(
                [
                    Conv2D(self.num_features * block, 3, strides=1, padding="same"),
                    BatchNormalization(),
                    LeakyReLU(alpha=0.1),
                    Conv2D(self.num_features * block, 3, strides=1, padding="same"),
                    BatchNormalization(),
                    LeakyReLU(alpha=0.1, name=f"down_block_relu_{i}"),
                    AveragePooling2D(pool_size=(2, 2), strides=2, padding="same") if i < len(self.blocks) - 1 else None,
                ]
            )

        # Define up-sampling layers
        self.up_stack = []
        for i, block in enumerate(self.blocks[:-1][::-1]):
            self.up_stack.append(
                [
                    UpSampling2D(size=(2, 2)),
                    Conv2D(self.num_features * block, 3, strides=1, padding="same"),
                    BatchNormalization(),
                    LeakyReLU(alpha=0.1),
                    Conv2D(self.num_features * block, 3, strides=1, padding="same"),
                    BatchNormalization(),
                    LeakyReLU(alpha=0.1),
                ]
            )

    def call(self, inputs, training=None):
        """
        Forward pass through the U-Net.
        """
        # Down-sampling path
        x = inputs
        skip_connections = []
        for i, block_layers in enumerate(self.down_stack):
            for layer in block_layers[:-1]:  # Skip the pooling layer in the last block
                x = layer(x, training=training)
            if block_layers[-1] is not None:  # Add skip connection and apply pooling if not the last block
                skip_connections.append(x)
                x = block_layers[-1](x)

        # Up-sampling path
        for i, (block_layers, skip) in enumerate(zip(self.up_stack, skip_connections[::-1])):
            x = block_layers[0](x)  # Up-sample
            x = concatenate([x, skip], axis=-1)  # Concatenate with skip connection
            for layer in block_layers[1:]:  # Apply convolutional layers
                x = layer(x, training=training)

        return x

def unet_global(image_size=(512, 512), num_classes=6, **kwargs):
    """
    Define the full U-Net model.
    """
    inputs = Input(shape=image_size + (3,))
    unet_layer_instance = UNet_layer(input_shape=image_size + (3,), **kwargs)
    extract_feature = unet_layer_instance(inputs)

    # Further processing
    x = Conv2D(filters=64, kernel_size=3, strides=1, padding="same")(extract_feature)
    x = LeakyReLU(alpha=0.1)(x)
    x = AveragePooling2D(2, strides=2, padding="same")(x)
    x = BatchNormalization()(x)

    x = Conv2D(filters=128, kernel_size=3, strides=1, padding="same")(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = AveragePooling2D(2, strides=2, padding="same")(x)
    x = BatchNormalization()(x)

    x = GlobalAveragePooling2D()(x)
    x = Dense(units=265, activation="relu")(x)
    x = Dropout(0.3)(x)
    global_map = Dense(units=num_classes, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=global_map)
    opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
    loss = tf.keras.losses.KLDivergence()

    model.compile(loss=loss, optimizer=opt)
    return model


def unet_hidden_crf(image_size=(512, 512), num_classes=6, pretrained_weights_path=None, **kwargs):
    inputs = Input(shape=image_size + (3,))

    unet_layer_instance = UNet_layer(input_shape=image_size + (3,), **kwargs)
    extract_feature = unet_layer_instance(inputs)

    feature_map = Conv2D(filters=num_classes, kernel_size=1, strides=1, padding="same")(extract_feature)
    feature_map = Activation('softmax', name='before_crf')(feature_map)

    x = Conv2D(filters=64, kernel_size=3, strides=1, padding="same")(extract_feature)
    x = LeakyReLU(alpha=0.1)(x)
    x = AveragePooling2D(2, strides=2, padding="same")(x)
    x = BatchNormalization()(x)

    x = Conv2D(filters=128, kernel_size=3, strides=1, padding="same")(x)
    x = LeakyReLU(alpha=0.1)(x)
    x = AveragePooling2D(2, strides=2, padding="same")(x)
    x = BatchNormalization()(x)

    x = GlobalAveragePooling2D()(x)
    x = Dense(units=265, activation="relu")(x)
    x = Dropout(0.3)(x)
    global_map = Dense(units=num_classes, activation="softmax", name="global_map")(x)

    
    crf = HiddenCrfRnnLayer(
        image_dims=image_size + (3,),
        num_classes=num_classes,
        theta_alpha=4.46,
        theta_beta=4.5,
        theta_gamma=0.11,
        num_iterations=1,
        trainable=True,
        name='hiddencrfrnn_trainable'
    )([feature_map, feature_map])

    # post_crf = Conv2D(filters=num_classes, kernel_size=1, strides=1, padding="same")(crf)

    log_global_map = tf.math.log(global_map + 1e-10)
    combined = Add(name="final_combination")([crf, log_global_map])
    outputs = Activation('softmax', name='final_result')(combined)

    model = Model(inputs=inputs, outputs=outputs)
    #Load pretrained weights selectively
    # if pretrained_weights_path:
    #     try:
    #         print(f"Loading pretrained weights from {pretrained_weights_path}...")
    #         model.load_weights(pretrained_weights_path)
    #     except Exception as e:
    #         print(f"Error loading pretrained weights: {e}")

    return model
# def unet_hidden_crf(image_size=(512, 512), num_classes=6, pretrained_weights_path=None, **kwargs):
#     """
#     Define the full U-Net model and optionally initialize parameters from a pretrained model without relying on layer names.
#     """
#     inputs = Input(shape=image_size + (3,))
    
#     # Initialize U-Net layers
#     unet_layer_instance = UNet_layer(input_shape=image_size + (3,), **kwargs)
#     extract_feature = unet_layer_instance(inputs)

#     # Further processing layers
#     x = Conv2D(filters=64, kernel_size=3, strides=1, padding="same")(extract_feature)
#     x = LeakyReLU(alpha=0.1)(x)
#     x = AveragePooling2D(2, strides=2, padding="same")(x)
#     x = BatchNormalization()(x)

#     x = Conv2D(filters=128, kernel_size=3, strides=1, padding="same")(x)
#     x = LeakyReLU(alpha=0.1)(x)
#     x = AveragePooling2D(2, strides=2, padding="same")(x)
#     x = BatchNormalization()(x)

#     x = GlobalAveragePooling2D()(x)
#     x = Dense(units=265, activation="relu")(x)
#     x = Dropout(0.3)(x)
#     global_map = Dense(units=num_classes, activation="softmax")(x)

#     # Add CRF layers
#     feature_map = Conv2D(filters=num_classes, kernel_size=1, strides=1, padding="same")(extract_feature)
#     feature_map = Activation('softmax', name='before_crf')(feature_map)
    
#     crf = HiddenCrfRnnLayer(
#         image_dims=image_size + (3, ),
#         num_classes=num_classes,
#         theta_alpha=4.46,
#         theta_beta=4.5,
#         theta_gamma=0.11,
#         num_iterations=1,
#         trainable=True,
#         name='hiddencrfrnn_trainable'
#     )([feature_map, feature_map, global_map])
    
#     # crf = CrfRnnLayer(
#     #     image_dims=image_size + (3, ),
#     #     num_classes=num_classes,
#     #     theta_alpha=4.46,
#     #     theta_beta=4.5,
#     #     theta_gamma=0.11,
#     #     num_iterations=1,
#     #     trainable=True,
#     #     name='hiddencrfrnn_trainable'
#     # )([feature_map, feature_map, global_map])

#     post_crf = Conv2D(filters=num_classes, kernel_size=1, strides=1, padding="same")(crf)
#     outputs = Activation('softmax', name='final_result')(post_crf)
    
#     model = Model(inputs=inputs, outputs=outputs)

#     # Load pretrained weights selectively
#     if pretrained_weights_path:
#         try:
#             print(f"Loading pretrained weights from {pretrained_weights_path}...")
#             pretrained_model = Model(inputs=inputs, outputs=global_map)
#             pretrained_model.load_weights(pretrained_weights_path)  # Load weights for layers up to `global_map`
            
#             # Extract pretrained weights
#             pretrained_weights = pretrained_model.get_weights()
#             model_weights = model.get_weights()
            
#             # Update weights for matching layers
#             for i in range(len(pretrained_weights)):
#                 if pretrained_weights[i].shape == model_weights[i].shape:
#                     model_weights[i] = pretrained_weights[i]
#                 else:
#                     print(f"Skipping layer {i}: shape mismatch {pretrained_weights[i].shape} vs {model_weights[i].shape}")
            
#             # Set updated weights back to the full model
#             model.set_weights(model_weights)
#             print("Pretrained weights successfully loaded for layers up to `global_map`.")
#         except Exception as e:
#             print(f"Error loading pretrained weights: {e}")
    
#     return model

### Train Schedule

In [ ]:
import math
LR_START = 1e-6
LR_MAX = 1e-3
LR_MIN = 1e-7
LR_RAMPUP_EPOCHS = 0
LR_SUSTAIN_EPOCHS = 0
EPOCHS = 50

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        decay_total_epochs = EPOCHS - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS - 1
        decay_epoch_index = epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS
        phase = math.pi * decay_epoch_index / decay_total_epochs
        cosine_decay = 0.5 * (1 + math.cos(phase))
        lr = (LR_MAX - LR_MIN) * cosine_decay + LR_MIN
    return lr

rng = [i for i in range(EPOCHS)]
lr_y = [lrfn(x) for x in rng]
plt.figure(figsize=(10, 4))
plt.plot(rng, lr_y, '-o')
plt.xlabel('epoch',size=14); plt.ylabel('learning rate',size=14)
plt.title('Cosine Training Schedule',size=16); plt.show()

LR2 = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

In [ ]:
USE_KAGGLE_SPECTROGRAMS = True
USE_EEG_SPECTROGRAMS = True
def build_model(base_model):

    inp = tf.keras.Input(shape=(128,256,8))
    # base_model = None 
    
    # RESHAPE INPUT 128x256x8 => 512x512x3 MONOTONE IMAGE
    # KAGGLE SPECTROGRAMS
    x1 = [inp[:,:,:,i:i+1] for i in range(4)]
    x1 = tf.keras.layers.Concatenate(axis=1)(x1)
    # EEG SPECTROGRAMS
    x2 = [inp[:,:,:,i+4:i+5] for i in range(4)]
    x2 = tf.keras.layers.Concatenate(axis=1)(x2)
    # MAKE 512X512X3
    if USE_KAGGLE_SPECTROGRAMS & USE_EEG_SPECTROGRAMS:
        x = tf.keras.layers.Concatenate(axis=2)([x1,x2])
    elif USE_EEG_SPECTROGRAMS: x = x2
    else: x = x1
    x = tf.keras.layers.Concatenate(axis=3)([x,x,x])
    # OUTPUT
    x = base_model(x)
    # x = tf.keras.layers.GlobalAveragePooling2D()(x)
    
    # x = tf.expand_dims(tf.reduce_sum(x, axis=0), axis=0)
    # x = tf.keras.layers.Dense(6,activation='softmax', dtype='float32')(x)
    
    # COMPILE MODEL
    model = tf.keras.Model(inputs=inp, outputs=x)
    opt = tf.keras.optimizers.Adam(learning_rate = 1e-3)
    loss = tf.keras.losses.KLDivergence()

    model.compile(loss=loss, optimizer = opt, metrics=[tf.keras.losses.KLDivergence()])

    return model

### Pretrain Unet

In [ ]:
from sklearn.model_selection import KFold, GroupKFold
import tensorflow.keras.backend as K, gc

LOAD_MODELS_FROM = './Models/'
VER = 1
NUM_EXPERTS_THRESHOLD = 10
# VER = 1
all_oof = []
all_true = []
train_data = train[train.expert_numner > NUM_EXPERTS_THRESHOLD]

gkf = GroupKFold(n_splits=5)
for i, (train_index, valid_index) in enumerate(gkf.split(train_data, train_data.target, train_data.patient_id)):  
    
    print('#'*25)
    print(f'### Fold {i+1}')
    
    train_gen = DataGenerator(train_data.iloc[train_index], shuffle=True, batch_size=32, augment=False)
    valid_gen = DataGenerator(train_data.iloc[valid_index], shuffle=False, batch_size=64, mode='valid')
    
    print(f'### train size {len(train_index)}, valid size {len(valid_index)}')
    print('#'*25)
    
    K.clear_session()       
    with strategy.scope():
        image_size = (512, 512)
        num_classes = 6
        model = unet_global(image_size = image_size, num_classes=num_classes, num_features=32, trainable=True)
        model = build_model(model)
    if LOAD_MODELS_FROM is None:
        model.fit(train_gen, verbose=1,
              validation_data = valid_gen,
              epochs=EPOCHS, callbacks = [LR2],)
        model.save_weights(f'Global_UNet_v{VER}_f{i}.h5')
    else:
        model.load_weights(f'{LOAD_MODELS_FROM}Global_UNet_v{VER}_f{i}.h5')
        model.evaluate(valid_gen)
        # model.save(f'{LOAD_MODELS_FROM}Global_UNet_v{VER}_f{i}_full.h5')
    oof = model.predict(valid_gen, verbose=1)
    all_oof.append(oof)
    all_true.append(train_data.iloc[valid_index][TARGETS].values)
    del model, oof
    gc.collect()
    
all_oof = np.concatenate(all_oof)
all_true = np.concatenate(all_true)

### Fine tune Hidden CRF layer

In [ ]:
import math
LR_START = 1e-6
LR_MAX = 1e-3
LR_MIN = 1e-7
LR_RAMPUP_EPOCHS = 0
LR_SUSTAIN_EPOCHS = 0
EPOCHS = 5

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        decay_total_epochs = EPOCHS - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS - 1
        decay_epoch_index = epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS
        phase = math.pi * decay_epoch_index / decay_total_epochs
        cosine_decay = 0.5 * (1 + math.cos(phase))
        lr = (LR_MAX - LR_MIN) * cosine_decay + LR_MIN
    return lr

rng = [i for i in range(EPOCHS)]
lr_y = [lrfn(x) for x in rng]
plt.figure(figsize=(10, 4))
plt.plot(rng, lr_y, '-o')
plt.xlabel('epoch',size=14); plt.ylabel('learning rate',size=14)
plt.title('Cosine Training Schedule',size=16); plt.show()

LR2 = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose = True)

In [ ]:
from sklearn.model_selection import KFold, GroupKFold
import tensorflow.keras.backend as K, gc

LOAD_MODELS_FROM = None
BATCH_SIZE = 1
VER = 2
NUM_EXPERTS_THRESHOLD = 10
all_oof = []
all_true = []
train_data = train[train.expert_numner > NUM_EXPERTS_THRESHOLD]

gkf = GroupKFold(n_splits=5)
for i, (train_index, valid_index) in enumerate(gkf.split(train_data, train_data.target, train_data.patient_id)):  
    
    print('#'*25)
    print(f'### Fold {i+1}')
    
    train_gen = DataGenerator(train_data.iloc[train_index], shuffle=True, batch_size=BATCH_SIZE, augment=False)
    valid_gen = DataGenerator(train_data.iloc[valid_index], shuffle=False, batch_size=BATCH_SIZE, mode='valid')
    
    print(f'### train size {len(train_index)}, valid size {len(valid_index)}')
    print('#'*25)
    
    K.clear_session()       
    with strategy.scope():
        image_size = (512, 512)
        num_classes = 6
        
        model = unet_hidden_crf(image_size = image_size, num_classes=num_classes, num_features=32, trainable=True)
        # model = unet_global(image_size = image_size, num_classes=num_classes, num_features=32, trainable=True)
        model = build_model(model, None)
    if LOAD_MODELS_FROM is None:
        model.fit(train_gen, verbose=1,
              validation_data = valid_gen,
              epochs=EPOCHS, callbacks = [LR2],)
        model.save_weights(f'Global_UNet_v{VER}_f{i}.h5')
    else:
        LOAD_MODELS_FROM = './Models/'
        model.load_weights(f'{LOAD_MODELS_FROM}Global_UNet_v{VER}_f{i}.h5')
        model.evaluate(valid_gen)
        # model.save(f'{LOAD_MODELS_FROM}Global_UNet_v{VER}_f{i}_full.h5')
    oof = model.predict(valid_gen, verbose=1)
    all_oof.append(oof)
    all_true.append(train_data.iloc[valid_index][TARGETS].values)
    del model, oof
    gc.collect()
    
all_oof = np.concatenate(all_oof)
all_true = np.concatenate(all_true)